In [5]:
from dask_image.imread import imread
import dask.array as da
import os
import numpy as np
import joblib
import dask  # Import Dask first
dask.config.set({'dataframe.query-planning': False})  # Disable query-planning

import dask.dataframe as dd  # Now import dask.dataframe
import pandas as pd
import dask.dataframe as dd
from spatialdata import read_zarr

c:\Users\Mattias\anaconda3\envs\classifier_env\lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(


In [6]:
sdata = read_zarr(r"C:\Users\Mattias\Documents\projects\HOGENT\Bach_proef\Bachelor_proef_VIB\Testdata_MACSima\Testdata_CA.zarr")

sdata

c:\Users\Mattias\anaconda3\envs\classifier_env\lib\site-packages\zarr\creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)


SpatialData object, with associated Zarr store: C:\Users\Mattias\Documents\projects\HOGENT\Bach_proef\Bachelor_proef_VIB\Testdata_MACSima\Testdata_CA.zarr
├── Images
│     ├── 'channel_0': DataArray[cyx] (1, 4500, 7000)
│     ├── 'channel_1': DataArray[cyx] (1, 4500, 7000)
│     ├── 'channel_2': DataArray[cyx] (1, 4500, 7000)
│     ├── 'channel_3': DataArray[cyx] (1, 4500, 7000)
│     ├── 'channel_4': DataArray[cyx] (1, 4500, 7000)
│     ├── 'channel_5': DataArray[cyx] (1, 4500, 7000)
│     ├── 'channel_6': DataArray[cyx] (1, 4500, 7000)
│     ├── 'channel_7': DataArray[cyx] (1, 4500, 7000)
│     ├── 'channel_8': DataArray[cyx] (1, 4500, 7000)
│     ├── 'channel_9': DataArray[cyx] (1, 4500, 7000)
│     ├── 'channel_10': DataArray[cyx] (1, 4500, 7000)
│     ├── 'channel_11': DataArray[cyx] (1, 4500, 7000)
│     ├── 'channel_12': DataArray[cyx] (1, 4500, 7000)
│     ├── 'channel_13': DataArray[cyx] (1, 4500, 7000)
│     ├── 'channel_14': DataArray[cyx] (1, 4500, 7000)
│     ├── 'channel_

In [14]:
IMAGE_SIZE = 256
STRIDE = IMAGE_SIZE - 32

def extract_patches(array:np.ndarray):
    patches = []
    h, w = array.shape

    # Ensure that the image dimensions are divisible by the patch size and stride
    for i in range(0, h - IMAGE_SIZE + 1, STRIDE):
        for j in range(0, w - IMAGE_SIZE + 1, STRIDE):
            img_patch = array[i:i+IMAGE_SIZE, j:j+IMAGE_SIZE]
            patches.append(img_patch)

    return np.array(patches)
    
extract_patches(sdata['channel_0'].data.squeeze()).shape

(589, 256, 256)

In [8]:
def extract_patches_per_channel(sdata, channel, patch_size=512, stride=256):
    """Extract patches from a specific channel in SpatialData."""
    img = sdata.images[f"channel_{channel}"].values[0] / 225
    labels = sdata.labels["labels"].values // 255

    patches, label_patches = [], []
    h, w = img.shape

    for i in range(0, h - patch_size, stride):
        count_x+=1
        for j in range(0, w - patch_size, stride):
            img_patch = array[i:i+patch_size, j:j+patch_size]

            patches.append(img_patch)

    print(count_x)
    return np.array(patches)

X = extract_patches(sdata['channel_0'].data.squeeze(), patch_size=512, stride=256)
len(X)

In [9]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, Input, Dropout
from tensorflow.keras.models import Model


In [10]:
def unet_model_smaller(input_size=(512, 512, 1)):
    inputs = Input(input_size)

    # Encoder
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)

    # Decoder
    u4 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c3)  # Match c2 filters
    u4 = concatenate([u4, c2])
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(u4)
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(c4)

    u5 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c4)  # Match c1 filters
    u5 = concatenate([u5, c1])
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(u5)
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(c5)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c5)

    model = Model(inputs, outputs)
    return model


In [11]:
def unet_model(input_size=(512, 512, 1)):
    inputs = Input(input_size)

    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = MaxPooling2D((2, 2))(c4)

    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    u6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1])
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs, outputs)
    return model


In [12]:
extract_patches_per_channel

<function __main__.extract_patches_per_channel(sdata, channel, patch_size=512, stride=256)>

In [13]:
batch_size = 8
epochs = 5

channels = 1

for channel in range(channels):
    print(f"\nTraining U-Net for Channel {channel}...\n")

    X, Y = extract_patches_per_channel(sdata, channel, patch_size=1024, stride=512)

    
    X = np.expand_dims(X, axis=-1)  
    Y = np.expand_dims(Y, axis=-1)  

    
    model = unet_model_smaller(input_size=(1024, 1024, 1))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(X, Y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

    model.save(f"unet_channel_{channel}.h5")

    print(f"\nFinished training U-Net for Channel {channel}!\n")


Training U-Net for Channel 0...

Epoch 1/5


ResourceExhaustedError: Graph execution error:

Detected at node 'model/conv2d_1/Relu' defined at (most recent call last):
    File "c:\Users\Mattias\anaconda3\envs\classifier_env\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\Mattias\anaconda3\envs\classifier_env\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\Users\Mattias\anaconda3\envs\classifier_env\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "c:\Users\Mattias\anaconda3\envs\classifier_env\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "c:\Users\Mattias\anaconda3\envs\classifier_env\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "c:\Users\Mattias\anaconda3\envs\classifier_env\lib\site-packages\tornado\platform\asyncio.py", line 205, in start
      self.asyncio_l

In [ ]:
new_model = tf.keras.models.load_model("unet_channel_0.h5")

X, Y =  extract_patches_per_channel(sdata, 2, patch_size=512, stride=256)


new_model.predict(X)

 3/13 ━━━━━━━━━━━━━━━━━━━━ 2:40 16s/step

In [19]:
IMAGE_SIZE = 256
STRIDE = IMAGE_SIZE - 32

from skimage.util import view_as_windows

def extract_patches_skimage(array: np.ndarray):
    window_shape = (IMAGE_SIZE, IMAGE_SIZE)
    step = STRIDE
    patches = view_as_windows(array, window_shape, step=step)
    patches = patches.reshape(-1, IMAGE_SIZE, IMAGE_SIZE)
    return patches


out = extract_patches_skimage(sdata['channel_0'].values.squeeze())
shape = extract_patches(sdata['channel_0'].data.squeeze())

In [18]:
out

array([[[12, 14, 10, ..., 20, 28, 39],
        [ 9,  7, 12, ..., 11, 28, 21],
        [ 0,  0,  8, ..., 22, 23,  7],
        ...,
        [ 2, 10, 16, ..., 39, 26, 41],
        [ 2,  5, 10, ..., 35, 24, 35],
        [ 8,  9, 10, ..., 40, 43, 23]],

       [[15, 20, 25, ..., 32, 34, 32],
        [26, 27, 24, ..., 27, 13, 22],
        [30, 34, 32, ..., 32, 12, 16],
        ...,
        [18, 22, 36, ..., 27, 28, 29],
        [15, 17, 28, ..., 29, 32, 19],
        [19, 17, 27, ..., 41, 46, 19]],

       [[63, 43, 42, ..., 18, 11, 10],
        [37, 25, 39, ..., 16,  6,  1],
        [33, 42, 45, ..., 18, 12,  8],
        ...,
        [33, 29, 24, ..., 36, 35, 36],
        [21, 17, 20, ..., 41, 44, 37],
        [17, 11,  9, ..., 27, 21, 16]],

       ...,

       [[49, 52, 78, ...,  2,  1,  1],
        [52, 53, 79, ...,  4,  9, 12],
        [32, 48, 71, ...,  9, 10, 18],
        ...,
        [13, 15, 22, ..., 12,  6, 13],
        [11, 13, 15, ...,  1,  5, 14],
        [10,  3,  4, ...,  0,  1

In [2]:
import sys
import os
import numpy as np
import logging
import keras

import gc
from tensorflow.keras import backend as K

from spatialdata import SpatialData
from spatialdata import read_zarr
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, Input
from tensorflow.keras.models import Model
from skimage.util import view_as_windows

c:\Users\Mattias\anaconda3\envs\classifier_env\lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(


In [3]:
IMAGE_SIZE = 256
STRIDE = IMAGE_SIZE - 32

def extract_patches_skimage(array):
    window_shape = (IMAGE_SIZE, IMAGE_SIZE)
    step = STRIDE
    patches = view_as_windows(array, window_shape, step=step)
    patches = patches.reshape(-1, IMAGE_SIZE, IMAGE_SIZE)
    
    return patches

In [4]:
import tensorflow as tf

model = tf.keras.models.load_model(r"..\model\temp\pixel_classifier_256.keras")

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  640         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 256, 256, 64  36928       ['conv2d[0][0]']                 
                                )                                                             

In [7]:
sdata = read_zarr(r"D:\test\2023_11_22_CelineEveraert_R1_B1_ROI1.zarr")

labels = extract_patches_skimage(sdata['annotations'].values) // 255
images = extract_patches_skimage(sdata['C-001_S-000_B_APC_R-01_W-B-1_ROI-01_A-None.tif'].values.squeeze()) / 255

input_image = images[0][..., np.newaxis]
input_image = np.expand_dims(input_image, axis=0) 

predictions = model.predict(input_image)

1/1 [==============================] - 6s 6s/step


In [ ]:
def dice_coef(y_true, y_pred, smooth=1):
    y_true_flat = y_true.flatten()
    y_pred_flat = y_pred.flatten()

    intersection = np.sum(y_true_flat * y_pred_flat)
    return (2. * intersection + smooth) / (np.sum(y_true_flat) + np.sum(y_pred_flat) + smooth)

def iou(y_true, y_pred, smooth=1):
    y_true_flat = y_true.flatten()
    y_pred_flat = y_pred.flatten()
    intersection = np.sum(y_true_flat * y_pred_flat)
    union = np.sum(y_true_flat) + np.sum(y_pred_flat) - intersection
    return (intersection + smooth) / (union + smooth)

def dice_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)


In [ ]:
dice_

In [ ]:
predictions = model.predict(input_image)

from PIL import Image
import numpy as np

img = Image.fromarray(predictions.squeeze(), 'L')
lab = Image.fromarray(labels[0])
lab.show()
img.show()

c:\Users\Mattias\anaconda3\envs\classifier_env\lib\site-packages\zarr\creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)


1/1 [==============================] - 0s 20ms/step


In [29]:
directories = open(r"..\model\files.txt", "r")

directories.read().split("\n")

['D:\\2024_03_15_CelineEveraert_2b_240321_001656_preprocessed\\2024_03_15_CelineEveraert_2b\\R01\\B01\\ROI1',
 'D:\\2024_03_15_CelineEveraert_2b_240321_001656_preprocessed\\2024_03_15_CelineEveraert_2b\\R01\\D01\\ROI1',
 'D:\\2024_03_15_CelineEveraert_3_240414_071301_preprocessed\\2024_03_15_CelineEveraert_3\\R01\\B01\\ROI1',
 'D:\\2024_03_15_CelineEveraert_3_240414_071301_preprocessed\\2024_03_15_CelineEveraert_3\\R01\\D01\\ROI1',
 'D:\\2024_03_15_CelineEveraert_3_240414_071301_preprocessed2\\2024_03_15_CelineEveraert_3\\R01\\B01\\ROI1',
 'D:\\2024_03_15_CelineEveraert_3_240414_071301_preprocessed2\\2024_03_15_CelineEveraert_3\\R01\\D01\\ROI1',
 'D:\\2024_08_29_M14_CelineEveraert_11_240911_171248_preprocessed\\2024_08_29_M14_CelineEveraert_11_240911_171248_preprocessed\\2024_08_29_M14_CelineEveraert_11\\R01\\B01\\ROI1',
 'D:\\2024_08_29_M14_CelineEveraert_11_240911_171248_preprocessed\\2024_08_29_M14_CelineEveraert_11_240911_171248_preprocessed\\2024_08_29_M14_CelineEveraert_11\\R01\\

In [10]:
def get_size(start_path = '.'):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            # skip if it is symbolic link
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)

    return total_size//1_000_000
list = []
dir = r"D:\data\train"
for arg in os.listdir(dir):
    size = get_size(os.path.join(dir, arg))
    list.append((arg, size))

def myFunc(e):
  return e[1]

list.sort(key=myFunc)

for i in list:
    print(f"{i[0]}: {i[1]}")

2024_09_12_M16_CellineEveraert_12_R01_B01_ROI2.zarr: 3854
2024_09_12_M16_CellineEveraert_12_R01_B01_ROI1.zarr: 3939
2024_09_06_M15_Annelies_Van Hemelryk_UCSC_Neuroblastoma_2_R01_A01_ROI1.zarr: 5596
Testdata_CA.zarr: 5997
2024_09_06_M15_Annelies_Van Hemelryk_UCSC_Neuroblastoma_2_R01_A01_ROI5.zarr: 6262
2024_09_20_M17_SPC-035_HelenaAegerter_R01_A01_ROI3.zarr: 6731
Testdata_CE.zarr: 7376
2024_03_15_CelineEveraert_3_R01_D01_ROI1.zarr: 7418
2024_09_12_M16_CellineEveraert_12_R01_B01_ROI3.zarr: 8077
2024_09_12_M16_CellineEveraert_12_R01_B01_ROI4.zarr: 8467
2024_03_15_CelineEveraert_3_R01_B01_ROI1.zarr: 9411
2024_09_20_M17_SPC-035_HelenaAegerter_R01_A01_ROI2.zarr: 10057
2024_09_06_M15_Annelies_Van Hemelryk_UCSC_Neuroblastoma_2_R01_A01_ROI4.zarr: 12542
2024-09-25_exp38_SPC-036_CelineEveraert 1_R01_B01_ROI2.zarr: 14028
2024_03_15_CelineEveraert_3_2_R01_D01_ROI1.zarr: 16447
2024_03_15_CelineEveraert_3_2_R01_B01_ROI1.zarr: 21533
2024_08_29_M14_CelineEveraert_11_R01_B01_ROI2.zarr: 22182
2024_08_29_